In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

In [33]:
alpha = 0.01

def one_hot_encode(Y):
    Y = Y.flatten()
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    return one_hot_Y

def softmax(Z, axis=0):
    exp = np.exp(Z)
    A = exp / np.sum(exp)
    print(np.sum(exp, axis=axis))
    return A

In [4]:
data = pd.read_csv("mnist_train.csv", header=None).to_numpy()
x, y = data[:, 1:], one_hot_encode(data[:, 0:1])  # x.shape, y.shape = (60000, 784) (60000, 10)
# plt.imshow(x[0].reshape(28, 28), cmap="gray")
# test  = pd.read_csv("mnist_test.csv", header=None).to_numpy()

In [ ]:
class Layer:
    def __init__(self, input, output, act_f="relu") -> None:
        self.input = input
        self.output = output
        self.w = np.random.rand(output, input)
        self.b = np.random.rand(output)
        # self.activation= act_f
        self.activation = self.relu if act_f == "relu" else self.softmax
        self.activation_ = self.relu_ if act_f == "relu" else self.softmax_
        self.alpha = alpha
        
    def relu(self, x):
        return np.maximum(x, 0)
    def relu_(self, x):
        return (x > 0) * 1

    def softmax(self, Z):
        A = np.exp(Z) / sum(np.exp(Z))
        return A
    def softmax_(self, Z):
        pass

    def forward(self, x):
        self.x = x
        self.z = x @ self.w.T + self.b
        self.a = self.activation(self.z)
        return self.a
    
    def backward(self, y):  # y.shape = (60000, 10)  Y is one hot encoded
        da = self.a - y
        dz = da * (self.z > 0)
        self.dw = self.x.T @ dz
        self.db = np.mean(dz, axis=0)
        self.dx = dz @ self.w.T
        self.update()
        return self.dx
    
    def update(self):
        self.w -= self.alpha * self.dw
        self.b -= self.alpha * self.db

# def forward_propagation(x, w, b):
    # a nural network with 2 hidden layers each layer containing 16 neurons
    # input layer contains 28*28 = 784 neurons
    # output layer contains 10 neurons

In [35]:
np.random.seed(2)
a = np.random.rand(2, 3).astype(np.float16)
print(a)
softmax(a)

[[0.436   0.02592 0.55   ]
 [0.4353  0.4204  0.3303 ]]
[3.094 2.549 3.125]


array([[0.1765 , 0.11707, 0.1978 ],
       [0.1764 , 0.1737 , 0.1588 ]], dtype=float16)